You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Trends, Players, and News

Introduction:
Artificial Intelligence (AI) has become a buzzword in the tech industry, with advancements and innovations constantly shaping the landscape. In this blog article, we will delve into the latest trends, key players, and noteworthy news in the field of AI.

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine Learning and Deep Learning algorithms are revolutionizing industries such as healthcare, finance, and tran

I now can give a great answer

Final Answer: 

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become a dominant force in the tech industry, driving innovation and transformation across various sectors. In this blog article, we will explore the latest trends, key players, and noteworthy news in the dynamic field of AI.

## Latest Trends in Artificial Intelligence
The rapid evolution of AI is evident in the increasing adoption of Machine Learning and Deep Learning algorithms. Industries like healthcare, finance, and transportation are leveraging these technologies to optimize processes, improve decision-making, and enhance customer experiences. Natural Language Processing (NLP) is another key trend, enabling businesses to communicate more effectively with customers through chatbots, virtual assistants, and sentiment analysis. Moreover, AI-powered automation is revolutionizing operations by streamlining workflows, reduc

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Trends, Players, and News

## Introduction
Artificial Intelligence (AI) has become a dominant force in the tech industry, driving innovation and transformation across various sectors. In this blog article, we will explore the latest trends, key players, and noteworthy news in the dynamic field of AI.

## Latest Trends in Artificial Intelligence
The rapid evolution of AI is evident in the increasing adoption of Machine Learning and Deep Learning algorithms. Industries like healthcare, finance, and transportation are leveraging these technologies to optimize processes, improve decision-making, and enhance customer experiences. Natural Language Processing (NLP) is another key trend, enabling businesses to communicate more effectively with customers through chatbots, virtual assistants, and sentiment analysis. Moreover, AI-powered automation is revolutionizing operations by streamlining workflows, reducing manual tasks, and driving efficiency gains across multiple sectors.

## Key Players in Artificial Intelligence
Several tech giants are at the forefront of AI innovation, shaping the landscape with their cutting-edge technologies and solutions. Google stands out with its AI-powered products such as Google Assistant and Google Translate, while Amazon leads in e-commerce and cloud services driven by AI. IBM is a key player in AI research and development, with its Watson platform driving advancements in natural language understanding and cognitive computing. Tesla is disrupting the automotive industry with its self-driving technology and AI integration in electric vehicles, setting new standards for safety and automation.

## Noteworthy News in Artificial Intelligence
Recent developments in AI have been making headlines, showcasing the remarkable capabilities of advanced technologies. OpenAI's GPT-3 model continues to impress with its language generation capabilities, pushing the boundaries of AI-generated content. Microsoft's acquisition of Nuance Communications for $19.7 billion underscores the growing importance of AI in healthcare, with AI-powered solutions transforming patient care and medical diagnostics. Facebook's AI research team is at the forefront of exploring ethical implications and responsible development practices in AI, emphasizing the need for transparency and accountability in the deployment of AI technologies.

## Conclusion
Artificial Intelligence is reshaping industries, driving innovation, and transforming the way we live and work. By staying informed about the latest trends, key players, and noteworthy news in AI, businesses and individuals can harness the power of AI to drive growth, efficiency, and competitive advantage. As the AI landscape continues to evolve, it is crucial to stay updated on the latest developments and opportunities in this dynamic field.

*Stay tuned for more insightful articles on specific AI topics and case studies to learn how you can leverage AI in your industry. Subscribe to our newsletter for the latest updates on Artificial Intelligence trends and news.*

By following this comprehensive content plan, we aim to provide valuable insights and information on Artificial Intelligence to our target audience, empowering them to make informed decisions and stay ahead in the rapidly evolving AI landscape.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Blog Article on "YOUR TOPIC HERE"

1. Latest Trends, Key Players, and Noteworthy News:
- Trend: The latest advancements in YOUR TOPIC HERE technology have revolutionized the industry.
- Key Players: Company A and Company B are leading the way in innovation within the YOUR TOPIC HERE sector.
- Noteworthy News: Recent studies have shown a significant increase in consumer interest in YOUR TOPIC HERE products.

2. Target Audience Analysis:
- Target Audience: The target audience for this blog article inc

I now can give a great answer

Final Answer:

# The Evolution of YOUR TOPIC HERE Technology: Trends, Players, and News

In today's fast-paced and tech-driven world, the advancements in YOUR TOPIC HERE technology have been nothing short of revolutionary. From cutting-edge innovations to groundbreaking developments, the industry continues to push the boundaries of what is possible. Companies like Company A and Company B have emerged as key players in driving innovation within the YOUR TOPIC HERE sector, setting the stage for a new era of possibilities.

The latest trends in YOUR TOPIC HERE technology are reshaping the way we interact with the world around us. Whether it's through enhanced user experiences, improved efficiency, or increased connectivity, these trends are shaping the future of the industry. Consumer interest in YOUR TOPIC HERE products is on the rise, with recent studies showing a significant increase in demand for these cutting-edge solutions. As the market continues to e

In [14]:
Markdown(result)

# The Evolution of YOUR TOPIC HERE Technology: Trends, Players, and News

In today's fast-paced and tech-driven world, the advancements in YOUR TOPIC HERE technology have been nothing short of revolutionary. From cutting-edge innovations to groundbreaking developments, the industry continues to push the boundaries of what is possible. Companies like Company A and Company B have emerged as key players in driving innovation within the YOUR TOPIC HERE sector, setting the stage for a new era of possibilities.

The latest trends in YOUR TOPIC HERE technology are reshaping the way we interact with the world around us. Whether it's through enhanced user experiences, improved efficiency, or increased connectivity, these trends are shaping the future of the industry. Consumer interest in YOUR TOPIC HERE products is on the rise, with recent studies showing a significant increase in demand for these cutting-edge solutions. As the market continues to evolve, staying informed about the latest trends and developments in YOUR TOPIC HERE technology is essential for tech enthusiasts, industry professionals, and consumers alike.

## Exploring Key Players and Industry News

Company A and Company B are at the forefront of innovation within the YOUR TOPIC HERE sector, driving forward with groundbreaking technologies and solutions. Their contributions to the industry have been instrumental in shaping the landscape of YOUR TOPIC HERE technology, setting new standards for excellence and pushing the boundaries of what is possible. By staying ahead of the curve and investing in research and development, these key players are paving the way for a brighter and more interconnected future.

Noteworthy news in the YOUR TOPIC HERE industry is shaping consumer preferences and market dynamics. From the latest product launches to industry partnerships and collaborations, these developments are influencing the way we interact with technology and the world around us. Understanding the impact of these changes on consumers and the market is essential for staying informed and making informed decisions in a rapidly evolving landscape. As the industry continues to evolve, keeping up with the latest news and developments is crucial for industry professionals and consumers interested in YOUR TOPIC HERE.

## Embracing Innovation and Looking Ahead

As we delve deeper into the world of YOUR TOPIC HERE technology, it becomes clear that the future holds endless possibilities for innovation and growth. By exploring the latest trends, profiling key players, and analyzing noteworthy news, we gain valuable insights into the industry's trajectory and potential impact on consumers and the market. Encouraging readers to explore more about YOUR TOPIC HERE by visiting our website or subscribing to our newsletter is a call to action for those looking to stay informed and ahead of the curve in this dynamic and ever-changing industry.

In conclusion, the evolution of YOUR TOPIC HERE technology continues to shape the way we live, work, and interact with the world around us. By staying informed about the latest trends, key players, and industry news, we can better navigate the complexities of this rapidly changing landscape and embrace the endless possibilities for innovation and growth. As we look towards the future, it is essential to stay curious, informed, and proactive in exploring the vast potential of YOUR TOPIC HERE technology.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).